In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier   # use MLPRegressor if regression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [19]:
all_features_data = pd.read_csv("/Users/suhaniagarwal/Downloads/all_features_data.csv")
all_features_change = all_features_data.drop(["temperature_2m_mean (°C)", "temperature_2m_min (°C)", "apparent_temperature_mean (°C)","apparent_temperature_min (°C)","apparent_temperature_max (°C)","wind_direction_10m_dominant (°)"], axis=1)

In [17]:
all_features_data

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),...,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_mean (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),wind_speed_10m_max (km/h),wind_gusts_10m_max (km/h),wind_direction_10m_dominant (°),Number of Admissions
0,2017-05-01,38.350,61.36,16.18,5.71,116.68,50.755,1.82,0.700,25.85,...,26.9,35.6,17.4,26.3,36.0,16.9,13.7,29.5,8,28.0
1,2017-05-02,48.600,77.75,15.48,5.74,116.68,50.755,4.74,0.900,43.53,...,29.4,37.8,19.4,27.7,38.1,16.9,17.2,29.5,46,14.0
2,2017-05-03,46.940,75.10,18.65,4.80,116.68,50.755,2.38,1.190,19.42,...,30.2,36.8,25.1,28.4,35.9,23.7,19.1,40.7,39,15.0
3,2017-05-04,57.095,119.76,15.73,14.73,116.68,50.755,1.82,0.645,11.61,...,29.2,36.0,23.6,28.1,37.4,21.9,18.8,35.3,20,17.0
4,2017-05-05,75.770,121.22,8.48,8.62,116.68,50.755,0.99,0.490,32.05,...,29.5,38.0,20.9,28.6,37.7,19.5,15.3,33.8,326,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2019-03-27,56.110,94.46,14.94,1.30,12.39,21.310,10.59,0.000,23.20,...,21.9,29.6,13.9,23.0,30.9,13.3,11.2,24.5,324,23.0
696,2019-03-28,28.690,73.86,27.43,9.46,32.22,43.840,9.62,0.000,40.74,...,23.8,30.6,18.0,25.4,32.1,18.9,11.1,25.6,332,41.0
697,2019-03-29,47.540,81.34,40.21,10.34,47.18,50.500,19.67,0.000,34.93,...,24.3,31.8,17.5,25.6,33.5,18.5,11.0,24.1,316,27.0
698,2019-03-30,52.130,88.82,65.08,10.78,66.81,66.040,37.93,0.000,36.24,...,24.0,32.2,18.3,24.6,34.9,17.1,21.7,44.6,334,13.0


In [10]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_fnn_rand = all_features_data["Number of Admissions"]

X_fnn_rand = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)


X_train_fnn_rand, X_test_fnn_rand, y_train_fnn_rand, y_test_fnn_rand = train_test_split(
    X_fnn_rand,
    y_fnn_rand,
    test_size=0.2,
    random_state=42,
    shuffle=True
)


fnn_rand = Pipeline([
    ("scaler", StandardScaler()),
    ("fnn", MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-4,
        learning_rate_init=0.001,
        max_iter=1000,
        early_stopping=True,
        random_state=42
    ))
])


fnn_rand.fit(X_train_fnn_rand, y_train_fnn_rand)
y_pred_fnn_rand = fnn_rand.predict(X_test_fnn_rand)

print("FNN (Random Split)")
print("R²   :", r2_score(y_test_fnn_rand, y_pred_fnn_rand))
print("RMSE:", np.sqrt(mean_squared_error(y_test_fnn_rand, y_pred_fnn_rand)))
print("MAE :", mean_absolute_error(y_test_fnn_rand, y_pred_fnn_rand))



FNN (Random Split)
R²   : 0.07327985254929115
RMSE: 7.544537746448478
MAE : 6.0100825255851875


In [11]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_fnn_time = all_features_data["Number of Admissions"]

X_fnn_time = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)


split_date = "2018-11-01"

train_idx = all_features_data["Timestamp"] < split_date
test_idx  = all_features_data["Timestamp"] >= split_date

X_train_fnn_time = X_fnn_time.loc[train_idx]
X_test_fnn_time  = X_fnn_time.loc[test_idx]

y_train_fnn_time = y_fnn_time.loc[train_idx]
y_test_fnn_time  = y_fnn_time.loc[test_idx]

fnn_time = Pipeline([
    ("scaler", StandardScaler()),
    ("fnn", MLPRegressor(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-4,
        learning_rate_init=0.001,
        max_iter=1000,
        early_stopping=True,
        random_state=42
    ))
])


fnn_time.fit(X_train_fnn_time, y_train_fnn_time)
y_pred_fnn_time = fnn_time.predict(X_test_fnn_time)

print("FNN (Time-Aware Split)")
print("R²   :", r2_score(y_test_fnn_time, y_pred_fnn_time))
print("RMSE:", np.sqrt(mean_squared_error(y_test_fnn_time, y_pred_fnn_time)))
print("MAE :", mean_absolute_error(y_test_fnn_time, y_pred_fnn_time))



FNN (Time-Aware Split)
R²   : -0.7639239269403026
RMSE: 9.846663851840074
MAE : 7.930891925680254


In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_svr = all_features_data["Number of Admissions"]

X_svr = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

X_train_svr, X_test_svr, y_train_svr, y_test_svr = train_test_split(
    X_svr, y_svr, test_size=0.2, random_state=42, shuffle=True
)


svr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svr", SVR(kernel="rbf", C=10, epsilon=0.1))
])

svr_pipeline.fit(X_train_svr, y_train_svr)

y_pred_svr = svr_pipeline.predict(X_test_svr)

print("SVR (Random Split)")
print("R²   :", r2_score(y_test_svr, y_pred_svr))
print("RMSE:", np.sqrt(mean_squared_error(y_test_svr, y_pred_svr)))
print("MAE :", mean_absolute_error(y_test_svr, y_pred_svr))


SVR (Random Split)
R²   : 0.13668994281443803
RMSE: 7.2818500804764925
MAE : 5.801893767372201


In [8]:
from sklearn.dummy import DummyRegressor


y_dummy = all_features_data["Number of Admissions"]

X_dummy = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

X_train_dummy, X_test_dummy, y_train_dummy, y_test_dummy = train_test_split(
    X_dummy, y_dummy, test_size=0.2, random_state=42, shuffle=True
)


dummy_reg = DummyRegressor(strategy="mean")
dummy_reg.fit(X_train_dummy, y_train_dummy)

y_pred_dummy = dummy_reg.predict(X_test_dummy)

print("Dummy Regressor (Random Split)")
print("R²   :", r2_score(y_test_dummy, y_pred_dummy))
print("RMSE:", np.sqrt(mean_squared_error(y_test_dummy, y_pred_dummy)))
print("MAE :", mean_absolute_error(y_test_dummy, y_pred_dummy))


Dummy Regressor (Random Split)
R²   : -0.0009045969974690315
RMSE: 7.840697074383431
MAE : 6.152040816326531


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_lin = all_features_data["Number of Admissions"]

X_lin = all_features_data.drop(
    columns=[
        "Number of Admissions",
        "Timestamp"
    ]
)


X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(
    X_lin,
    y_lin,
    test_size=0.2,
    random_state=42,
    shuffle=True
)


lin_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("linreg", LinearRegression())
])


lin_pipeline.fit(X_train_lin, y_train_lin)


y_pred_lin = lin_pipeline.predict(X_test_lin)

print("Linear Regression (Random Split)")
print("--------------------------------")
print("R²   :", r2_score(y_test_lin, y_pred_lin))
print("RMSE:", np.sqrt(mean_squared_error(y_test_lin, y_pred_lin)))
print("MAE :", mean_absolute_error(y_test_lin, y_pred_lin))


Linear Regression (Random Split)
--------------------------------
R²   : 0.12584311318987063
RMSE: 7.327452721434663
MAE : 5.824296997025744


In [12]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_xgb_rand = all_features_data["Number of Admissions"]

X_xgb_rand = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

X_train_xgb_rand, X_test_xgb_rand, y_train_xgb_rand, y_test_xgb_rand = train_test_split(
    X_xgb_rand, y_xgb_rand, test_size=0.2, random_state=42, shuffle=True
)


xgb_rand = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    objective="reg:squarederror",
    random_state=42
)

xgb_rand.fit(X_train_xgb_rand, y_train_xgb_rand)
y_pred_xgb_rand = xgb_rand.predict(X_test_xgb_rand)

print("XGBoost (Random Split)")
print("R²   :", r2_score(y_test_xgb_rand, y_pred_xgb_rand))
print("RMSE:", np.sqrt(mean_squared_error(y_test_xgb_rand, y_pred_xgb_rand)))
print("MAE :", mean_absolute_error(y_test_xgb_rand, y_pred_xgb_rand))


XGBoost (Random Split)
R²   : 0.21273346014993866
RMSE: 6.953752515306717
MAE : 5.563463538033622


In [13]:

y_xgb_time = all_features_data["Number of Admissions"]

X_xgb_time = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

split_date = "2018-11-01"

train_idx = all_features_data["Timestamp"] < split_date
test_idx  = all_features_data["Timestamp"] >= split_date

X_train_xgb_time = X_xgb_time.loc[train_idx]
X_test_xgb_time  = X_xgb_time.loc[test_idx]

y_train_xgb_time = y_xgb_time.loc[train_idx]
y_test_xgb_time  = y_xgb_time.loc[test_idx]


xgb_time = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    objective="reg:squarederror",
    random_state=42
)

xgb_time.fit(X_train_xgb_time, y_train_xgb_time)
y_pred_xgb_time = xgb_time.predict(X_test_xgb_time)

print("XGBoost (Time-Aware Split)")
print("R²   :", r2_score(y_test_xgb_time, y_pred_xgb_time))
print("RMSE:", np.sqrt(mean_squared_error(y_test_xgb_time, y_pred_xgb_time)))
print("MAE :", mean_absolute_error(y_test_xgb_time, y_pred_xgb_time))


XGBoost (Time-Aware Split)
R²   : -0.19350324256615248
RMSE: 8.099551815472129
MAE : 6.584219269405137


In [14]:
from sklearn.ensemble import RandomForestRegressor


y_rf_rand = all_features_data["Number of Admissions"]

X_rf_rand = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

X_train_rf_rand, X_test_rf_rand, y_train_rf_rand, y_test_rf_rand = train_test_split(
    X_rf_rand, y_rf_rand, test_size=0.2, random_state=42, shuffle=True
)


rf_rand = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_rand.fit(X_train_rf_rand, y_train_rf_rand)
y_pred_rf_rand = rf_rand.predict(X_test_rf_rand)

print("Random Forest (Random Split)")
print("R²   :", r2_score(y_test_rf_rand, y_pred_rf_rand))
print("RMSE:", np.sqrt(mean_squared_error(y_test_rf_rand, y_pred_rf_rand)))
print("MAE :", mean_absolute_error(y_test_rf_rand, y_pred_rf_rand))


Random Forest (Random Split)
R²   : 0.227951248165697
RMSE: 6.886216867937668
MAE : 5.464008620326992


In [15]:

y_rf_time = all_features_data["Number of Admissions"]

X_rf_time = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)

X_train_rf_time = X_rf_time.loc[train_idx]
X_test_rf_time  = X_rf_time.loc[test_idx]

y_train_rf_time = y_rf_time.loc[train_idx]
y_test_rf_time  = y_rf_time.loc[test_idx]


rf_time = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_time.fit(X_train_rf_time, y_train_rf_time)
y_pred_rf_time = rf_time.predict(X_test_rf_time)

print("Random Forest (Time-Aware Split)")
print("R²   :", r2_score(y_test_rf_time, y_pred_rf_time))
print("RMSE:", np.sqrt(mean_squared_error(y_test_rf_time, y_pred_rf_time)))
print("MAE :", mean_absolute_error(y_test_rf_time, y_pred_rf_time))


Random Forest (Time-Aware Split)
R²   : -0.18877831574571102
RMSE: 8.083503370984662
MAE : 6.5292611488385575


In [25]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


y_xgb_rand_c = all_features_data["Number of Admissions"]

X_xgb_rand_c = all_features_data.drop(
    columns=[
    "Number of Admissions",
    "Timestamp",
    "Index Value",
    "NO (µg/m³)",
    "NOx (ppb)",
    "NH3 (µg/m³)",
    "Benzene (µg/m³)",
    "Toluene (µg/m³)",
    "temperature_2m_min (°C)",
    "apparent_temperature_mean (°C)",
    "apparent_temperature_max (°C)",
    "apparent_temperature_min (°C)",
    "wind_gusts_10m_max (km/h)"
]
)

X_train_xgb_rand_c, X_test_xgb_rand_c, y_train_xgb_rand_c, y_test_xgb_rand_c = train_test_split(
    X_xgb_rand_c, y_xgb_rand_c, test_size=0.2, random_state=42, shuffle=True
)


xgb_rand_c = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    objective="reg:squarederror",
    random_state=42
)

xgb_rand_c.fit(X_train_xgb_rand_c, y_train_xgb_rand_c)
y_pred_xgb_rand_c = xgb_rand_c.predict(X_test_xgb_rand_c)

print("XGBoost (Random Split)")
print("R²   :", r2_score(y_test_xgb_rand_c, y_pred_xgb_rand_c))
print("RMSE:", np.sqrt(mean_squared_error(y_test_xgb_rand_c, y_pred_xgb_rand_c)))
print("MAE :", mean_absolute_error(y_test_xgb_rand_c, y_pred_xgb_rand_c))


XGBoost (Random Split)
R²   : 0.16470116762671538
RMSE: 7.16274137553682
MAE : 5.62397997038705


In [24]:
print(all_features_data.columns.tolist())


['Timestamp', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO (µg/m³)', 'NO2 (µg/m³)', 'NOx (ppb)', 'NH3 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'Ozone (µg/m³)', 'Benzene (µg/m³)', 'Toluene (µg/m³)', 'Index Value', 'temperature_2m_mean (°C)', 'temperature_2m_max (°C)', 'temperature_2m_min (°C)', 'apparent_temperature_mean (°C)', 'apparent_temperature_max (°C)', 'apparent_temperature_min (°C)', 'wind_speed_10m_max (km/h)', 'wind_gusts_10m_max (km/h)', 'wind_direction_10m_dominant (°)', 'Number of Admissions']


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# =========================
# Target
# =========================
y_rf_rand_c = all_features_data["Number of Admissions"]

# =========================
# Features (same dropped columns as XGBoost)
# =========================
X_rf_rand_c = all_features_data.drop(
    columns=[
        "Number of Admissions",
        "Timestamp",
        "Index Value",
        "NO (µg/m³)",
        "NOx (ppb)",
        "NH3 (µg/m³)",
        "Benzene (µg/m³)",
        "Toluene (µg/m³)",
        "temperature_2m_min (°C)",
        "apparent_temperature_mean (°C)",
        "apparent_temperature_max (°C)",
        "apparent_temperature_min (°C)",
        "wind_gusts_10m_max (km/h)"
    ]
)

# =========================
# Wind direction encoding (same idea, new vars)
# =========================
X_rf_rand_c["wind_dir_sin"] = np.sin(
    np.deg2rad(X_rf_rand_c["wind_direction_10m_dominant (°)"])
)
X_rf_rand_c["wind_dir_cos"] = np.cos(
    np.deg2rad(X_rf_rand_c["wind_direction_10m_dominant (°)"])
)

X_rf_rand_c = X_rf_rand_c.drop(
    columns=["wind_direction_10m_dominant (°)"]
)

# =========================
# Train-test split (random, same as XGB)
# =========================
X_train_rf_rand_c, X_test_rf_rand_c, y_train_rf_rand_c, y_test_rf_rand_c = train_test_split(
    X_rf_rand_c,
    y_rf_rand_c,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# =========================
# Random Forest model
# =========================
rf_rand_c = RandomForestRegressor(
    n_estimators=300,
    max_depth=4,              # matches XGB tree depth
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=0.85,        # analogous to colsample_bytree
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

# =========================
# Train & predict
# =========================
rf_rand_c.fit(X_train_rf_rand_c, y_train_rf_rand_c)
y_pred_rf_rand_c = rf_rand_c.predict(X_test_rf_rand_c)

# =========================
# Evaluation (same metrics)
# =========================
print("Random Forest (Random Split)")
print("R² :", r2_score(y_test_rf_rand_c, y_pred_rf_rand_c))
print("RMSE:", np.sqrt(mean_squared_error(y_test_rf_rand_c, y_pred_rf_rand_c)))
print("MAE :", mean_absolute_error(y_test_rf_rand_c, y_pred_rf_rand_c))


Random Forest (Random Split)
R² : 0.16925717905630489
RMSE: 7.143180620303536
MAE : 5.665833728524794


In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# =========================
# Target
# =========================
y_rf_rand_c = all_features_data["Number of Admissions"]

# =========================
# Features (drop redundant + wind direction)
# =========================
X_rf_rand_c = all_features_data.drop(
    columns=[
        "Number of Admissions",
        "Timestamp",
        "Index Value",
        "NO (µg/m³)",
        "NOx (ppb)",
        "NH3 (µg/m³)",
        "Benzene (µg/m³)",
        "Toluene (µg/m³)",
        "temperature_2m_min (°C)",
        "apparent_temperature_mean (°C)",
        "apparent_temperature_max (°C)",
        "apparent_temperature_min (°C)",
        "wind_gusts_10m_max (km/h)",
        "wind_direction_10m_dominant (°)"
    ]
)

# =========================
# Train-test split (random)
# =========================
X_train_rf_rand_c, X_test_rf_rand_c, y_train_rf_rand_c, y_test_rf_rand_c = train_test_split(
    X_rf_rand_c,
    y_rf_rand_c,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# =========================
# Random Forest model
# =========================
rf_rand_c = RandomForestRegressor(
    n_estimators=300,
    max_depth=4,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=0.85,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

# =========================
# Train & predict
# =========================
rf_rand_c.fit(X_train_rf_rand_c, y_train_rf_rand_c)
y_pred_rf_rand_c = rf_rand_c.predict(X_test_rf_rand_c)

# =========================
# Evaluation
# =========================
print("Random Forest (Random Split, No Wind Direction)")
print("R² :", r2_score(y_test_rf_rand_c, y_pred_rf_rand_c))
print("RMSE:", np.sqrt(mean_squared_error(y_test_rf_rand_c, y_pred_rf_rand_c)))
print("MAE :", mean_absolute_error(y_test_rf_rand_c, y_pred_rf_rand_c))


Random Forest (Random Split, No Wind Direction)
R² : 0.1724285358265929
RMSE: 7.129533053936597
MAE : 5.684488434260923


In [28]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# =========================
# Target
# =========================
y_xgb_rand_c = all_features_data["Number of Admissions"]

# =========================
# Features (drop redundant)
# =========================
X_xgb_rand_c = all_features_data.drop(
    columns=[
        "Number of Admissions",
        "Timestamp",
        "Index Value",
        "NO (µg/m³)",
        "NOx (ppb)",
        "NH3 (µg/m³)",
        "Benzene (µg/m³)",
        "Toluene (µg/m³)",
        "temperature_2m_min (°C)",
        "apparent_temperature_mean (°C)",
        "apparent_temperature_max (°C)",
        "apparent_temperature_min (°C)",
        "wind_gusts_10m_max (km/h)"
    ]
)

# =========================
# Wind direction encoding
# =========================
X_xgb_rand_c["wind_dir_sin"] = np.sin(
    np.deg2rad(X_xgb_rand_c["wind_direction_10m_dominant (°)"])
)
X_xgb_rand_c["wind_dir_cos"] = np.cos(
    np.deg2rad(X_xgb_rand_c["wind_direction_10m_dominant (°)"])
)

X_xgb_rand_c = X_xgb_rand_c.drop(
    columns=["wind_direction_10m_dominant (°)"]
)

# =========================
# Train-test split (random)
# =========================
X_train_xgb_rand_c, X_test_xgb_rand_c, y_train_xgb_rand_c, y_test_xgb_rand_c = train_test_split(
    X_xgb_rand_c,
    y_xgb_rand_c,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

# =========================
# XGBoost model (same as yours)
# =========================
xgb_rand_c = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.85,
    colsample_bytree=0.85,
    objective="reg:squarederror",
    random_state=42
)

# =========================
# Train & predict
# =========================
xgb_rand_c.fit(X_train_xgb_rand_c, y_train_xgb_rand_c)
y_pred_xgb_rand_c = xgb_rand_c.predict(X_test_xgb_rand_c)

# =========================
# Evaluation
# =========================
print("XGBoost (Random Split, With Wind Direction)")
print("R² :", r2_score(y_test_xgb_rand_c, y_pred_xgb_rand_c))
print("RMSE:", np.sqrt(mean_squared_error(y_test_xgb_rand_c, y_pred_xgb_rand_c)))
print("MAE :", mean_absolute_error(y_test_xgb_rand_c, y_pred_xgb_rand_c))


XGBoost (Random Split, With Wind Direction)
R² : 0.15215327022090808
RMSE: 7.216340349184611
MAE : 5.72918575831822


In [29]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# =========================
# Target
# =========================
y_xgb_cv5_rand_c = all_features_data["Number of Admissions"]

# =========================
# Features (drop redundant + wind direction)
# =========================
X_xgb_cv5_rand_c = all_features_data.drop(
    columns=[
        "Number of Admissions",
        "Timestamp",
        "Index Value",
        "NO (µg/m³)",
        "NOx (ppb)",
        "NH3 (µg/m³)",
        "Benzene (µg/m³)",
        "Toluene (µg/m³)",
        "temperature_2m_min (°C)",
        "apparent_temperature_mean (°C)",
        "apparent_temperature_max (°C)",
        "apparent_temperature_min (°C)",
        "wind_gusts_10m_max (km/h)",
        "wind_direction_10m_dominant (°)"
    ]
)

# =========================
# 5-Fold Cross-Validation setup
# =========================
kf_xgb_cv5_rand_c = KFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

# =========================
# Storage for metrics
# =========================
r2_scores_xgb_cv5_rand_c = []
rmse_scores_xgb_cv5_rand_c = []
mae_scores_xgb_cv5_rand_c = []

# =========================
# CV loop
# =========================
for fold_idx_xgb_cv5_rand_c, (train_idx_xgb_cv5_rand_c, test_idx_xgb_cv5_rand_c) in enumerate(
    kf_xgb_cv5_rand_c.split(X_xgb_cv5_rand_c), start=1
):
    X_train_xgb_cv5_rand_c = X_xgb_cv5_rand_c.iloc[train_idx_xgb_cv5_rand_c]
    X_test_xgb_cv5_rand_c  = X_xgb_cv5_rand_c.iloc[test_idx_xgb_cv5_rand_c]

    y_train_xgb_cv5_rand_c = y_xgb_cv5_rand_c.iloc[train_idx_xgb_cv5_rand_c]
    y_test_xgb_cv5_rand_c  = y_xgb_cv5_rand_c.iloc[test_idx_xgb_cv5_rand_c]

    model_xgb_cv5_rand_c = xgb.XGBRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        objective="reg:squarederror",
        random_state=42
    )

    model_xgb_cv5_rand_c.fit(X_train_xgb_cv5_rand_c, y_train_xgb_cv5_rand_c)

    y_pred_xgb_cv5_rand_c = model_xgb_cv5_rand_c.predict(
        X_test_xgb_cv5_rand_c
    )

    # Metrics
    r2_scores_xgb_cv5_rand_c.append(
        r2_score(y_test_xgb_cv5_rand_c, y_pred_xgb_cv5_rand_c)
    )
    rmse_scores_xgb_cv5_rand_c.append(
        np.sqrt(mean_squared_error(y_test_xgb_cv5_rand_c, y_pred_xgb_cv5_rand_c))
    )
    mae_scores_xgb_cv5_rand_c.append(
        mean_absolute_error(y_test_xgb_cv5_rand_c, y_pred_xgb_cv5_rand_c)
    )

    print(
        f"Fold {fold_idx_xgb_cv5_rand_c} | "
        f"R²={r2_scores_xgb_cv5_rand_c[-1]:.3f}, "
        f"RMSE={rmse_scores_xgb_cv5_rand_c[-1]:.3f}, "
        f"MAE={mae_scores_xgb_cv5_rand_c[-1]:.3f}"
    )

# =========================
# Mean CV performance
# =========================
print("\nXGBoost – 5 Fold CV (Random, No Wind Direction)")
print(f"Mean R²  : {np.mean(r2_scores_xgb_cv5_rand_c):.3f}")
print(f"Mean RMSE: {np.mean(rmse_scores_xgb_cv5_rand_c):.3f}")
print(f"Mean MAE : {np.mean(mae_scores_xgb_cv5_rand_c):.3f}")


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Fold 1 | R²=0.210, RMSE=6.966, MAE=5.510
Fold 2 | R²=0.093, RMSE=7.282, MAE=5.802
Fold 3 | R²=0.069, RMSE=7.455, MAE=5.673
Fold 4 | R²=0.100, RMSE=6.849, MAE=5.513
Fold 5 | R²=0.085, RMSE=6.882, MAE=5.422

XGBoost – 5 Fold CV (Random, No Wind Direction)
Mean R²  : 0.112
Mean RMSE: 7.087
Mean MAE : 5.584


In [31]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# =========================
# Target
# =========================
y_xgb_cv5_all_c = all_features_data["Number of Admissions"]

# =========================
# Features (ALL columns except target)
# =========================
X_xgb_cv5_all_c = all_features_data.drop(
    columns=["Number of Admissions", "Timestamp"]
)


# =========================
# 5-Fold Cross-Validation setup
# =========================
kf_xgb_cv5_all_c = KFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

# =========================
# Storage for metrics
# =========================
r2_scores_xgb_cv5_all_c = []
rmse_scores_xgb_cv5_all_c = []
mae_scores_xgb_cv5_all_c = []

# =========================
# CV loop
# =========================
for fold_idx_xgb_cv5_all_c, (
    train_idx_xgb_cv5_all_c,
    test_idx_xgb_cv5_all_c
) in enumerate(kf_xgb_cv5_all_c.split(X_xgb_cv5_all_c), start=1):

    X_train_xgb_cv5_all_c = X_xgb_cv5_all_c.iloc[train_idx_xgb_cv5_all_c]
    X_test_xgb_cv5_all_c  = X_xgb_cv5_all_c.iloc[test_idx_xgb_cv5_all_c]

    y_train_xgb_cv5_all_c = y_xgb_cv5_all_c.iloc[train_idx_xgb_cv5_all_c]
    y_test_xgb_cv5_all_c  = y_xgb_cv5_all_c.iloc[test_idx_xgb_cv5_all_c]

    model_xgb_cv5_all_c = xgb.XGBRegressor(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.85,
        colsample_bytree=0.85,
        objective="reg:squarederror",
        random_state=42
    )

    model_xgb_cv5_all_c.fit(
        X_train_xgb_cv5_all_c,
        y_train_xgb_cv5_all_c
    )

    y_pred_xgb_cv5_all_c = model_xgb_cv5_all_c.predict(
        X_test_xgb_cv5_all_c
    )

    # Metrics
    r2_scores_xgb_cv5_all_c.append(
        r2_score(y_test_xgb_cv5_all_c, y_pred_xgb_cv5_all_c)
    )
    rmse_scores_xgb_cv5_all_c.append(
        np.sqrt(mean_squared_error(y_test_xgb_cv5_all_c, y_pred_xgb_cv5_all_c))
    )
    mae_scores_xgb_cv5_all_c.append(
        mean_absolute_error(y_test_xgb_cv5_all_c, y_pred_xgb_cv5_all_c)
    )

    print(
        f"Fold {fold_idx_xgb_cv5_all_c} | "
        f"R²={r2_scores_xgb_cv5_all_c[-1]:.3f}, "
        f"RMSE={rmse_scores_xgb_cv5_all_c[-1]:.3f}, "
        f"MAE={mae_scores_xgb_cv5_all_c[-1]:.3f}"
    )

# =========================
# Mean CV performance
# =========================
print("\nXGBoost – 5 Fold CV (Random, ALL Features)")
print(f"Mean R²  : {np.mean(r2_scores_xgb_cv5_all_c):.3f}")
print(f"Mean RMSE: {np.mean(rmse_scores_xgb_cv5_all_c):.3f}")
print(f"Mean MAE : {np.mean(mae_scores_xgb_cv5_all_c):.3f}")


Fold 1 | R²=0.237, RMSE=6.847, MAE=5.434
Fold 2 | R²=0.120, RMSE=7.175, MAE=5.724
Fold 3 | R²=0.070, RMSE=7.455, MAE=5.760
Fold 4 | R²=0.122, RMSE=6.767, MAE=5.353
Fold 5 | R²=0.089, RMSE=6.864, MAE=5.650

XGBoost – 5 Fold CV (Random, ALL Features)
Mean R²  : 0.127
Mean RMSE: 7.021
Mean MAE : 5.584
